In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [2]:
data = pd.read_parquet("stock_data.parquet")
stock_names = data['comnam'].unique()

repeat = True
while repeat:
    count = 0
    dfs = []
    sizes = []
    relevant_stocks = np.random.choice(stock_names,size=10, replace=False)
    for stock in relevant_stocks:
        subset = data[data['comnam'] == stock]
        df = subset[['prc']]
        df.columns = [stock]
        df.index = pd.to_datetime(subset['date'])
        dfs.append(df)
        sizes.append(df.shape[0])
    dfs = [x for _,x in reversed(sorted(zip(sizes,dfs), key = lambda x:x[0]))]
    final = dfs[0]
    for i in range(1, 10):
        final = final.join(dfs[i])
    final.index.name = 'date'
    final = final.groupby(by='date').mean()
    final = final.interpolate()
    period = np.random.randint(30, final.shape[0])
    final = final.iloc[period-30:period]
    if final.iloc[0].isna().any() or final.iloc[29].isna().any():
        repeat = True
    else:
        repeat = False

Create an initial portfolio

In [3]:
initial_budget = 100000
initial_prices = final.iloc[0].to_numpy()
available_money = initial_budget * .95
P_0 = np.zeros(10) #initial portfolio
break_bool = True
ignore =  np.random.randint(0,10)
while break_bool:
    stock_buy = np.random.randint(0,10)
    price = initial_prices[stock_buy]
    if stock_buy != ignore:
        if (available_money * .75) - price >= 0:
            available_money = available_money - price
            P_0[stock_buy] = P_0[stock_buy] + 1
        else:
            break_bool = False

w_0 = initial_budget * .05 + available_money
print(w_0)
print(P_0)
final_prices = final.iloc[29].to_numpy()
available_money = initial_budget * .9
break_bool = True
P_t = np.zeros(10) #target portfolio

while break_bool:
    stock_buy = np.random.randint(0,10)
    price = final_prices[stock_buy]
    if available_money - price >= 0:
        available_money = available_money - price
        P_t[stock_buy] = P_t[stock_buy] + 1
    else:
        break_bool = False
print(P_t)

5276.611358642578
[57. 61. 54. 61. 59.  0. 60. 68. 65. 60.]
[44. 45. 36. 44. 49. 51. 57. 56. 44. 53.]


# Formulations

## With Perfect Information

In [4]:
F = 2.99  * np.ones(10)# trading fees
# P_0 = starting portfolio
# P_t = target portfolio
# w_0 = starting cash
# w_f = final free cash
budget = 20000
M = initial_budget



m = gp.Model()
# Define Variables
P_b = m.addMVar((10,), vtype=GRB.INTEGER)
P_s = m.addMVar((10,), vtype=GRB.INTEGER)
z_s = m.addMVar((10,), vtype=GRB.BINARY)
z_b = m.addMVar((10,), vtype=GRB.BINARY)
# Constraints

## Meet Target
m.addConstr(
    P_0 + P_b - P_s >= P_t
)

## Have Money
m.addConstr(
    w_0 + initial_prices @ P_s - initial_prices @ P_b - F @ z_s - F @ z_b >=0
)

## if we sell a stock, we pay a trading price
m.addConstr(
    M*(1-z_s) >= P_s
)

## If we buy a stock, we pay a trading fee

m.addConstr(
    M*(1-z_b) >= P_b
)

# Set objective: Maximize end portfolio value
m.ModelSense = GRB.MAXIMIZE
m.setObjective(
    - F @ z_s - F @ z_b + final_prices @ (P_0 + P_b - P_s)
)

m.optimize()
final_portfolio = P_0 + P_b.x - P_s.x
free_cash = w_0 + initial_prices @ P_s.x - initial_prices @ P_b.x
final_portfolio_value = free_cash + final_prices @ final_portfolio
print(f"Final Portfolio Value: {final_portfolio_value}")
print(f"Free Cash: {free_cash}")
print(f"Final Portfolio: {final_portfolio}")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-20
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 31 rows, 40 columns and 100 nonzeros
Model fingerprint: 0x7d33d619
Variable types: 0 continuous, 40 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [3e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+05]
Presolve removed 30 rows and 30 columns
Presolve time: 0.02s
Presolved: 1 rows, 10 columns, 10 nonzeros
Variable types: 0 continuous, 10 integer (0 binary)
Found heuristic solution: objective 100529.27943

Root relaxation: objective 1.010785e+05, 1 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl 

## Multi-stage

##

In [10]:
F = 2.99  * np.ones(10)# trading fees
prices = final.interpolate().to_numpy()
M = initial_budget
# P_0 = starting portfolio
# P_t = target portfolio
# w_0 = starting cash

m = gp.Model()

# Define Variables
P_b = m.addMVar((30,10), vtype=GRB.INTEGER)
P_s = m.addMVar((30,10), vtype=GRB.INTEGER)
z_s = m.addMVar((30,10), vtype=GRB.BINARY)
z_b = m.addMVar((30,10), vtype=GRB.BINARY)
P_time = m.addMVar((30,10), vtype=GRB.INTEGER)
w = m.addMVar((30,), vtype=GRB.CONTINUOUS, lb = 0)
# Constraints

## Represent Portfolios across time

### Time step 0
m.addConstr(
    P_time[0,:] == P_0 + P_b[0,:] - P_s[0,:]
)
### Rest of the time
m.addConstrs((
    P_time[i,:] == P_time[i-1,:] - P_b[i,:] - P_s[i,:] for i in range(1,30)
))

### Final_target
m.addConstr(
    P_time[29,:] >= P_t
)

## Represent money
### First time step
m.addConstr(
    w[0] == w_0 - prices[0,:] @ P_b[0,:] + prices[0,:] @ P_s[0,:] - F @ z_b[0,:] - F @ z_s[0,:]
)
## Rest of the time
m.addConstrs((
    w[i] == w[i-1] - prices[i,:] @ P_b[i,:] + prices[i,:] @ P_s[i,:] - F @ z_b[i,:] - F @ z_s[i,:] for i in range(1,30)
))

## Trading fees
### if we sell a stock, we pay a trading price
m.addConstr(
    M*(1-z_s) >= P_s
)

### If we buy a stock, we pay a trading fee
m.addConstr(
    M*(1-z_b) >= P_b
)

# Maximize end Portfolio value
m.ModelSense = GRB.MAXIMIZE
m.setObjective(
    w[29] + prices[29,:] @ P_time[29,:]
)
m.optimize()
final_portfolio = P_time[29,:].x
free_cash = w[29].x
final_portfolio_value = free_cash + prices[29,:] @ final_portfolio
print(f"Final Portfolio Value: {final_portfolio_value}")
print(f"Free Cash: {free_cash}")
print(f"Final Portfolio: {final_portfolio}")

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 940 rows, 1530 columns and 3659 nonzeros
Model fingerprint: 0x6964cc6c
Variable types: 30 continuous, 1500 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e+00, 7e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+01, 1e+05]
Presolve removed 302 rows and 302 columns
Presolve time: 0.02s
Presolved: 638 rows, 1228 columns, 3052 nonzeros
Variable types: 30 continuous, 1198 integer (599 binary)

Root relaxation: objective 1.013008e+05, 18 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 101300.850    0    2          - 1